In [69]:
#get_ipython().magic('reset -sf')

# Models

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import neighbors
from sklearn import feature_selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
import time
import dalex as dx
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
from torch.autograd import Variable 
from captum.attr import (
    IntegratedGradients,
    GradientShap,
    DeepLift,
    DeepLiftShap,
    LayerConductance,
    NeuronConductance,
    NoiseTunnel,
)
import lime.lime_tabular
import pickle
import matplotlib.pyplot as plt

First, I import pre-cleaned data, after I divide the dataset in training and testing, 60% and 40% respectively. Everything has to be scaled and I prepare the folds for the 5 cross validation. The steps I am going to follow in wach model are the following:
- Recursive feature elimination, I am going to remove the non important features
- Grid search, in order to try all the hyperparameters and choose the best ones
- Create model with the hyperparameters selected previously and fit it
- Make the predictions
- Shapley values to understand the model and the predictions

In [2]:
data_info_original = pd.read_csv('info_data.csv')
data_info_original = data_info_original.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_info = pd.read_csv('info_datav2.csv')
data_info = data_info.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_gaussian = data_info[data_info.ANO_FACTURA == 2020]

C:\Users\cul_g\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data_info_original.shape

(1073878, 27)

In [4]:
def groupData(df, col_groups, ref_col):
    testing =  df.groupby(col_groups)[ref_col].sum()
    grouped_testing = []
    for k,v in zip(testing.index, testing.values):
        val = [k[0],k[1],k[2],v,k[3], k[4],k[5],k[6],k[7],k[8],k[9],k[10],k[11],k[12],k[13],k[14],k[15],k[16],k[17],k[18],k[19],k[20],k[21],k[22],k[23],k[24], k[25]]
        grouped_testing.append(val)
    grouped_testing = pd.DataFrame(grouped_testing, columns = df.columns)
    #grouped_testing = grouped_testing.reset_index(drop=True)
    return grouped_testing

def getISOCountry(iso_country):
    indexes = data_info_original[data_info_original['NUMERO_DEUDOR_PAIS_ID'] == iso_country].index
    iso_code = data_info.iloc[[indexes[0]]]['NUMERO_DEUDOR_PAIS_ID']
    return iso_code.values[0]

def getValuesFilter(ds, value, columns, target):
    if value == '*': #no filter 
        return ds
    indexes = getIndexFilter(ds, value, target)
    datat = getValues(ds, indexes, columns)
    return datat

def getIndexFilter(dt, value, target):
    indexes = dt[dt[target] == value].index
    return indexes

def getValues(dt, indexes, columns):
    datat = []
    for k, v in zip(dt.index, dt.values):
        if k in indexes:
            datat.append(v)
    df = pd.DataFrame(datat, columns = columns)
    return df

def managementDataFile(dat, name_file, r_w):      
    # Its important to use binary mode
    if r_w == 'r':
        dbfile = open(name_file, 'ab')
        pickle.dump(dat, dbfile)
    elif r_w == 'w':
        dbfile = open(name_file, 'rb')
        db = pickle.load(dbfile)
    dbfile.close()

In [5]:
columns = ['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA', 'TEMPORADA_COMERCIAL_ID', 'PRODUCTO_ID', 'TALLA', 'ESFUERZO_VENTA_ID', 'NUMERO_DEUDOR_PAIS_ID', 'JERARQUIA_PROD_ID', 'GRUPO_ARTICULO_PRODUCTO_ID', 'GENERO_PRODUCTO', 'CATEGORIA', 'TIPOLOGIA', 'CONSUMER_COLOR', 'CREMALLERA', 'CORDONES', 'OUTSOLE_SUELA_TIPO', 'OUTSOLE_SUELA_SUBTIPO', 'PLANTILLA_EXTRAIBLE', 'CONTACTO_SN', 'EDAD_SN', 'GENERO_CONTACTO', 'EDAD_COMPRA', 'EDAD_RANGO_COMPRA', 'CIUDAD_CONTACTO', 'IDIOMA_CONTACTO']

In [6]:
data_info_grouped = groupData(data_info, columns, 'IMP_VENTA_NETO_EUR')
data_info_grouped = data_info_grouped.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_info_original_grouped = groupData(data_info_original, columns, 'IMP_VENTA_NETO_EUR')
data_info_original_grouped = data_info_original_grouped.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])

In [7]:
iso_spain = getISOCountry('ES') # Get iso from a country you want  to
data_info_filtered_grouped = getValuesFilter(data_info_grouped, '*', data_info_grouped.columns, 'NUMERO_DEUDOR_PAIS_ID') # iso = * -> no filter by country
data_info_filtered_grouped = data_info_filtered_grouped.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_info_original_filtered_grouped = getValuesFilter(data_info_original_grouped, '*', data_info_original_grouped.columns, 'NUMERO_DEUDOR_PAIS_ID') # iso = * -> no filter by country
data_info_original_filtered_grouped = data_info_original_filtered_grouped.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])

In [8]:
# Filter data by country
#iso_spain = getISOCountry('ES') # Get iso from a country you want  to
#data_info_filtered = getValuesFilter(data_info, '*', data_info.columns, 'NUMERO_DEUDOR_PAIS_ID') # iso = * -> no filter by numero deudor pais id
#data_info_filtered = data_info_filtered.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
#data_info_original_filtered = getValuesFilter(data_info_original, '*', data_info_original.columns, 'NUMERO_DEUDOR_PAIS_ID') # iso = * -> no filter by numero deudor pais id
#data_info_original_filtered = data_info_original_filtered.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])

In [9]:
traindataset_original, testdataset_original = train_test_split(data_info_original_filtered_grouped, test_size=0.4, shuffle= False) # To use all the data, change to -> data_info
traindataset, testdataset = train_test_split(data_info_filtered_grouped, test_size=0.4, shuffle= False) # To use all the data, change to -> data_info
x_train = traindataset.loc[:, traindataset.columns != 'IMP_VENTA_NETO_EUR']
y_train = traindataset.loc[:, traindataset.columns == 'IMP_VENTA_NETO_EUR']
x_train = x_train.drop(columns=['EDAD_RANGO_COMPRA'])
x_test = testdataset.loc[:, testdataset.columns != 'IMP_VENTA_NETO_EUR']
y_test = testdataset.loc[:, testdataset.columns == 'IMP_VENTA_NETO_EUR']
x_test = x_test.drop(columns = 'EDAD_RANGO_COMPRA')

In [10]:
#traindataset_original, testdataset_original = train_test_split(data_info_original_filtered, test_size=0.4, shuffle= False) # To use all the data, change to -> data_info
#traindataset, testdataset = train_test_split(data_info_filtered, test_size=0.4, shuffle= False) # To use all the data, change to -> data_info
#x_train = traindataset.loc[:, traindataset.columns != 'IMP_VENTA_NETO_EUR']
#y_train = traindataset.loc[:, traindataset.columns == 'IMP_VENTA_NETO_EUR']
#x_train = x_train.drop(columns=['EDAD_RANGO_COMPRA'])
#x_test = testdataset.loc[:, testdataset.columns != 'IMP_VENTA_NETO_EUR']
#y_test = testdataset.loc[:, testdataset.columns == 'IMP_VENTA_NETO_EUR']
#x_test = x_test.drop(columns = 'EDAD_RANGO_COMPRA')

In [11]:
#scaler = StandardScaler()
#x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
#x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)
normalizer = MinMaxScaler(feature_range = (-1, 1))
x_train = pd.DataFrame(normalizer.fit_transform(x_train), columns= x_train.columns, index = traindataset.index)
x_test = pd.DataFrame(normalizer.fit_transform(x_test), columns= x_test.columns, index = testdataset.index)
mseresults = pd.DataFrame()
timeexecution = pd.DataFrame()

In [12]:
folds = KFold(n_splits = 5, shuffle = False) # if shuffle false, random state doesn't matter

## Linear Model

I have to choose which are the most important features in the model, I will use R2 to get it.

In [95]:
hyper_params = [{'n_features_to_select': list(range(1, 26))}]
lm = LinearRegression()
rfe = RFE(lm)
model_cv = GridSearchCV(estimator = rfe, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1, return_train_score=True)      
model_cv.fit(x_train, y_train)
cv_results = pd.DataFrame(model_cv.cv_results_)

In [ ]:
display(cv_results)
print(model_cv.best_params_)

In [ ]:
#plt.figure(figsize=(16,6))
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='lower right')
plt.savefig('../Output/testscoretrain.png')
plt.show()

display(cv_results[['param_n_features_to_select', 'mean_train_score', 'mean_test_score']])

In [13]:
n_features_optimal = 25 # model_cv.best_params_['n_features_to_select']
lm = LinearRegression()
rfe = RFE(lm, n_features_to_select= n_features_optimal)             
rfe = rfe.fit(x_train, y_train)

I select only the parameters I am interested in

In [14]:
def removeFeatures(dt_train, dt_test, rfe_model):
    chosen = pd.DataFrame(rfe_model.support_, index=dt_train.columns, columns=['Rank'])
    featuresnotselected = []
    for k, v in zip(chosen.index, chosen.values):
        if v == False:
            featuresnotselected.append(k)
    dt_train_final = dt_train.drop(columns= featuresnotselected)
    dt_test_final = dt_test.drop(columns= featuresnotselected)
    return dt_train_final, dt_test_final

In [15]:
x_train_LM, x_test_LM = removeFeatures(x_train, x_test, rfe)

Finally, I create the model with the best parametrization possible

In [16]:
lm_model = LinearRegression()
start_time = time.time()
lm_model.fit(x_train_LM, y_train)
timeexecution['lm'] = (time.time() - start_time)
y_pred = lm_model.predict(x_test_LM)
results = y_test.copy()
results['lm'] = y_pred

In [17]:
display(results)

,IMP_VENTA_NETO_EUR,lm
641700,110.31,110.577613
641701,110.31,111.485371
641702,110.31,112.607074
641703,80.38,121.092480
641704,86.57,102.172864
...,...,...
1069495,82.68,154.071804
1069496,56.81,143.517590
1069497,56.81,147.638070
1069498,68.19,244.240802


#De moment no ho utilitzar�
F-test, p-values:

In [18]:
f_val, p_val = feature_selection.f_regression(x_train_LM, y_train) #Repassar
display(list(zip(x_train_LM.columns, p_val)))
print('')
display(list(zip(x_train_LM.columns, f_val)))

D:\Python\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[('ANO_FACTURA', 4.1434055778222634e-231),
 ('MES_FACTURA', 0.0),
 ('FECHA_FACTURA', 0.0),
 ('TEMPORADA_COMERCIAL_ID', 0.0),
 ('PRODUCTO_ID', 2.3894921765629806e-06),
 ('TALLA', 0.0),
 ('ESFUERZO_VENTA_ID', 0.0),
 ('NUMERO_DEUDOR_PAIS_ID', 0.0),
 ('JERARQUIA_PROD_ID', 0.0),
 ('GRUPO_ARTICULO_PRODUCTO_ID', 0.0),
 ('GENERO_PRODUCTO', 0.0),
 ('CATEGORIA', 0.0),
 ('TIPOLOGIA', 0.0),
 ('CONSUMER_COLOR', 0.0),
 ('CREMALLERA', 0.0),
 ('CORDONES', 0.0),
 ('OUTSOLE_SUELA_TIPO', 0.0),
 ('OUTSOLE_SUELA_SUBTIPO', 0.0),
 ('PLANTILLA_EXTRAIBLE', 0.0),
 ('CONTACTO_SN', 0.530563606061178),
 ('EDAD_SN', 0.0),
 ('GENERO_CONTACTO', 1.336425876962243e-119),
 ('EDAD_COMPRA', 1.953740549724508e-273),
 ('CIUDAD_CONTACTO', 0.0),
 ('IDIOMA_CONTACTO', 1.2182858071684708e-178)]

[('ANO_FACTURA', 1054.4040344097423),
 ('MES_FACTURA', 13736.605663469945),
 ('FECHA_FACTURA', 2603.4289561131222),
 ('TEMPORADA_COMERCIAL_ID', 5333.720285900557),
 ('PRODUCTO_ID', 22.253691562766942),
 ('TALLA', 43307.2983188973),
 ('ESFUERZO_VENTA_ID', 4150.099866883849),
 ('NUMERO_DEUDOR_PAIS_ID', 9793.690289166436),
 ('JERARQUIA_PROD_ID', 65292.98199596666),
 ('GRUPO_ARTICULO_PRODUCTO_ID', 120259.19322822579),
 ('GENERO_PRODUCTO', 48086.87504027428),
 ('CATEGORIA', 33732.95716462647),
 ('TIPOLOGIA', 16193.592980515188),
 ('CONSUMER_COLOR', 4962.795860924913),
 ('CREMALLERA', 13283.941855771413),
 ('CORDONES', 2026.706006740628),
 ('OUTSOLE_SUELA_TIPO', 4827.620341551089),
 ('OUTSOLE_SUELA_SUBTIPO', 7479.131152984338),
 ('PLANTILLA_EXTRAIBLE', 5197.039935901332),
 ('CONTACTO_SN', 0.39331239868704987),
 ('EDAD_SN', 4564.633761783719),
 ('GENERO_CONTACTO', 540.9154228734118),
 ('EDAD_COMPRA', 1249.5051631407036),
 ('CIUDAD_CONTACTO', 4689.334473132314),
 ('IDIOMA_CONTACTO', 812.686471

In [19]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: ', metrics.r2_score(y_test, y_pred))
mseresults['lm'] = metrics.mean_squared_error(y_test, y_pred)

Mean Absolute Error: 68.98225742153356
Mean Squared Error: 6340.82948686885
Root Mean Squared Error: 79.6293255457363
R2:  -2.05915698685919


In [20]:
plt.barh(range(0, 25), lm_model.coef_[0])
plt.yticks(range(0, 25), x_train_LM.columns, fontsize = 8)
plt.title('Coefficients')
plt.savefig('../Output/coefflinear.png')
plt.show()

In [20]:
variable_groups = {
    'Date': ['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA', 'TEMPORADA_COMERCIAL_ID'],
    'Product': ['PRODUCTO_ID', 'TALLA', 'GRUPO_ARTICULO_PRODUCTO_ID', 'GENERO_PRODUCTO', 'CATEGORIA', 'TIPOLOGIA', 'CONSUMER_COLOR', 'CREMALLERA', 'CORDONES', 'OUTSOLE_SUELA_TIPO', 'OUTSOLE_SUELA_SUBTIPO', 'PLANTILLA_EXTRAIBLE'],
    'Age': ['EDAD_SN', 'EDAD_COMPRA'],
    'ContanctInfo': ['NUMERO_DEUDOR_PAIS_ID', 'CIUDAD_CONTACTO', 'IDIOMA_CONTACTO', 'GENERO_CONTACTO', 'CONTACTO_SN'],
    'SalePerson': ['ESFUERZO_VENTA_ID']
}

In [22]:
linear_explainer = dx.Explainer(lm_model, x_test_LM, y_test)
explanation = linear_explainer.model_parts()
explanation.plot()
groupedexpl = linear_explainer.model_parts(variable_groups=variable_groups)
groupedexpl.plot()

In [21]:
indexspaindata = getIndexFilter(testdataset, 14, 'NUMERO_DEUDOR_PAIS_ID')
indexspaindata_original = getIndexFilter(testdataset_original, 'ES', 'NUMERO_DEUDOR_PAIS_ID')
usa_iso = getISOCountry('US')
indexusadata = getIndexFilter(testdataset, usa_iso, 'NUMERO_DEUDOR_PAIS_ID')
indexusadata_original = getIndexFilter(testdataset_original, 'US', 'NUMERO_DEUDOR_PAIS_ID')

In [24]:
indexspaindata[0]

Spain data

In [27]:
display(testdataset_original.loc[indexspaindata_original[0]])
display(testdataset_original.loc[indexspaindata_original[20]])
display(testdataset_original.loc[indexspaindata_original[200]])
display(testdataset_original.loc[indexspaindata_original[500]])

In [28]:
bd_henry_es_0 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexspaindata[0]]]), type = 'shap', B= 100)
bd_henry_es_0.plot()
bd_henry_es_1 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexspaindata[20]]]), type = 'shap', B= 100)
bd_henry_es_1.plot()
bd_henry_es_2 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexspaindata[200]]]), type = 'shap', B= 100)
bd_henry_es_2.plot()
bd_henry_es_3 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexspaindata[500]]]), type = 'shap', B= 100)
bd_henry_es_3.plot()

US data

In [29]:
display(testdataset_original.loc[indexusadata_original[0]])
display(testdataset_original.loc[indexusadata_original[20]])
display(testdataset_original.loc[indexusadata_original[200]])
display(testdataset_original.loc[indexusadata_original[500]])

In [30]:
bd_henry_us_1 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexusadata[0]]]), type = 'shap', B= 100)
bd_henry_us_1.plot()
bd_henry_us_2 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexusadata[20]]]), type = 'shap', B= 100)
bd_henry_us_2.plot()
bd_henry_us_3 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexusadata[200]]]), type = 'shap', B= 100)
bd_henry_us_3.plot()
bd_henry_us_4 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexusadata[500]]]), type = 'shap', B= 100)
bd_henry_us_4.plot()

## XGBoost Model

For this model, I am going to work with, the parameters max_depth, learning_rate and subsample from the Xgboost model and n_features_to_select from RFE.

In [ ]:
dtrain = xgb.DMatrix(x_train, label=y_train, feature_names=list(x_train.columns))
dtest = xgb.DMatrix(x_test, feature_names=list(x_test.columns))

In [13]:
#hyper_params = {'estimator__max_depth':[1, 10, 15], 'estimator__n_estimators': [150, 300, 500], 'estimator__learning_rate':[0.1, 0.05, 0.01] }#, 'n_features_to_select': list(range(1, 26))}
hyper_params = {'n_features_to_select': list(range(1, 26))}
xgbm = xgb.XGBRegressor(learning_rate =0.01, n_estimators=215, max_depth=10, min_child_weight=0.8, subsample=1, nthread=4)
rfe_xgboost = RFE(xgbm)
model_cv_xgb = GridSearchCV(estimator = rfe_xgboost, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1, return_train_score=True)      
model_cv_xgb.fit(x_train, y_train)
cv_results_xgb = pd.DataFrame(model_cv_xgb.cv_results_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [ ]:
display(cv_results_xgb)
print(model_cv_xgb.best_params_)

In [ ]:
#plt.figure(figsize=(16,6))
plt.plot(cv_results_xgb["param_n_features_to_select"], cv_results_xgb["mean_test_score"])
plt.plot(cv_results_xgb["param_n_features_to_select"], cv_results_xgb["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='lower right')
plt.savefig('../Output/testscoretrain_xgb.png')
plt.show()

display(cv_results_xgb[['param_n_features_to_select', 'mean_train_score', 'mean_test_score']])

Once I get the best parametrization, I execute the model with it, by the moment this is an example because I haven't execute the previous, too long.

In [22]:
n_features_optimal = 21 # model_cv.best_params_['n_features_to_select']
xgbm = xgb.XGBRegressor(learning_rate =0.01, n_estimators=215, max_depth=10, min_child_weight=0.8, subsample=1, nthread=4)
rfe = RFE(xgbm, n_features_to_select= n_features_optimal)             
rfe = rfe.fit(x_train, y_train)

In [23]:
x_train_XGBM, x_test_XGBM = removeFeatures(x_train, x_test, rfe)

In [24]:
display('Featues chosen: ', set(x_train_XGBM.columns))
display('Features discarted: ',set(x_train.columns) - set(x_train_XGBM.columns))

'Featues chosen: '

{'ANO_FACTURA',
 'CATEGORIA',
 'CIUDAD_CONTACTO',
 'CONSUMER_COLOR',
 'CORDONES',
 'CREMALLERA',
 'EDAD_COMPRA',
 'ESFUERZO_VENTA_ID',
 'FECHA_FACTURA',
 'GENERO_PRODUCTO',
 'IDIOMA_CONTACTO',
 'JERARQUIA_PROD_ID',
 'MES_FACTURA',
 'NUMERO_DEUDOR_PAIS_ID',
 'OUTSOLE_SUELA_SUBTIPO',
 'OUTSOLE_SUELA_TIPO',
 'PLANTILLA_EXTRAIBLE',
 'PRODUCTO_ID',
 'TALLA',
 'TEMPORADA_COMERCIAL_ID',
 'TIPOLOGIA'}

'Features discarted: '

{'CONTACTO_SN', 'EDAD_SN', 'GENERO_CONTACTO', 'GRUPO_ARTICULO_PRODUCTO_ID'}

In [25]:
xgbm = xgb.XGBRegressor(learning_rate =0.01, n_estimators=215, max_depth=10, min_child_weight=0.8, subsample=1, nthread=4)
start_time = time.time()
xgbm.fit(x_train_XGBM, y_train)
timeexecution['xgboost'] = (time.time() - start_time)
y_pred = xgbm.predict(x_test_XGBM)

In [26]:
#results = y_test.copy()
results['xgboost'] = y_pred

In [27]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))
mseresults['xgboost'] = metrics.mean_squared_error(y_test, y_pred)

Mean Absolute Error: 21.714228681888905
Mean Squared Error: 901.8878491721711
Root Mean Squared Error: 30.031447670270094
R2: 0.564880821843251


Firstly, I create an explainer for the model, the inputs are the model, x_train and y_train. After, i can get the variable importance and the reverse cumulative distribution of residuals

In [28]:
variable_groups = {
    'Date': ['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA', 'TEMPORADA_COMERCIAL_ID'],
    'Product': ['PRODUCTO_ID', 'TALLA', 'GENERO_PRODUCTO', 'CATEGORIA', 'TIPOLOGIA', 'CONSUMER_COLOR', 'CREMALLERA', 'CORDONES', 'OUTSOLE_SUELA_TIPO', 'OUTSOLE_SUELA_SUBTIPO', 'PLANTILLA_EXTRAIBLE'],
    'Age': ['EDAD_COMPRA'],
    'ContanctInfo': ['NUMERO_DEUDOR_PAIS_ID', 'CIUDAD_CONTACTO', 'IDIOMA_CONTACTO'],
    'SalePerson': ['ESFUERZO_VENTA_ID']
}

In [29]:
xgboost_explainer = dx.Explainer(xgbm, x_test_XGBM, y_test)
explanation = xgboost_explainer.model_parts()
explanation.plot()
res = xgboost_explainer.model_performance(model_type='regression')
res.plot()
groupedexpl = xgboost_explainer.model_parts(variable_groups=variable_groups)
groupedexpl.plot()

Preparation of a new explainer is initiated

  -> data              : 427800 rows 21 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 427800 values
  -> model_class       : xgboost.sklearn.XGBRegressor (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x000002B58D904E50> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 29.2, mean = 1.05e+02, max = 3.2e+02
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.5e+02, mean = 2.31, max = 7.01e+02
  -> model_info        : package xgboost

A new explainer has been created!


In [30]:
print(xgboost_explainer.model_performance().result)

                     mse       rmse        r2        mae        mad
XGBRegressor  901.887849  30.031448  0.564881  21.714229  16.755986


Also, I can group the features into a new ones. In this case, I have created different variables that include a similar meaning.
- Date: ANO_FACTURA, MES_FACTURA, FECHA_FACTURA, TEMPORADA_COMERCIAL_ID
- Product: PRODUCTO_ID, TALLA, GRUPO_ARTICULO_PRODUCTO_ID, GENERO_PRODUCTO, CATEGORIA, TIPOLOGIA, CONSUMER_COLOR, CREMALLERA, CORDONES, OUTSOLE_SUELA_TIPO, OUTSOLE_SUELA_SUBTIPO, PLANTILLA_EXTRAIBLE
- Age: EDAD_SN, EDAD_COMPRA
- ContanctInfo: NUMERO_DEUDOR_PAIS_ID, CIUDAD_CONTACTO, IDIOMA_CONTACTO, GENERO_CONTACTO, CONTACTO_SN
- SalePerson: ESFUERZO_VENTA_ID

In [31]:
xgb_sh_es_0 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexspaindata[0]]]), type = 'shap')
xgb_sh_es_0.plot()
xgb_sh_es_1 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexspaindata[20]]]), type = 'shap')
xgb_sh_es_1.plot()
xgb_sh_es_2 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexspaindata[200]]]), type = 'shap')
xgb_sh_es_2.plot()
xgb_sh_es_3 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexspaindata[500]]]), type = 'shap')
xgb_sh_es_3.plot()

NameError: name 'xgb_sh_es_0' is not defined

In [ ]:
xgb_sh_us_0 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexusadata[0]]]), type = 'shap')
xgb_sh_us_0.plot()
xgb_sh_us_1 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexusadata[20]]]), type = 'shap')
xgb_sh_us_1.plot()
xgb_sh_us_2 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexusadata[200]]]), type = 'shap')
xgb_sh_us_2.plot()
xgb_sh_us_3 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexusadata[500]]]), type = 'shap')
xgb_sh_us_3.plot()

In [ ]:
#plt.figure(figsize=(16,6))
#chosen = pd.DataFrame(rfe_xgboost.support_, index=x_train.columns, columns=['Rank'])
#featuresnotselected = []
#for k, v in zip(chosen.index, chosen.values):
#    if v == False:
#        featuresnotselected.append(k)
#x_train_final = x_train.drop(columns= featuresnotselected)
#x_test_final = x_test.drop(columns= featuresnotselected)

#plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
#plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
#plt.xlabel('number of features')
#plt.ylabel('r-squared')
#plt.title("Optimal Number of Features")
#plt.legend(['test score', 'train score'], loc='upper left')

## k-nearest neighbour regression

In [ ]:
rmse_val = [] #to store rmse values for different k
mse_val = []
model = neighbors.KNeighborsRegressor(n_neighbors = 7)
start_time = time.time()
model.fit(x_train, y_train)  #fit the model
timeexecution['knn'] = (time.time() - start_time)
y_pred=model.predict(x_test) #make prediction on test set
mse = metrics.mean_squared_error(y_test, y_pred)
error = np.sqrt(metrics.mean_squared_error(y_test, y_pred)) #calculate rmse
rmse_val.append(error) #store rmse values
mse_val.append(mse)
results['knn'] = y_pred
mseresults['knn'] = metrics.mean_squared_error(y_test, y_pred)

In [ ]:
#curvermse = pd.DataFrame(rmse_val) #elbow curve 
#curvermse.plot()
#curvemse = pd.DataFrame(mse_val)
#curvemse.plot()

## NN

### MLP Regressor

In [25]:
nn_reg = MLPRegressor(hidden_layer_sizes=(300, 300),  activation='logistic', solver='adam', alpha=0.01, batch_size='auto', learning_rate='constant', learning_rate_init=0.01, max_iter=1000, shuffle=False, tol=0.0001, verbose=True, early_stopping= True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
nn_reg.fit(x_train, np.ravel(y_train))
y_pred = nn_reg.predict(x_test)
nn_reg.score(x_test, y_test)

In [26]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))

In [75]:
nn_reg_pred = lambda x: nn_reg.predict(x).astype(float)

In [ ]:
results['nn'] = y_pred

In [130]:
indexspaindata = getIndexFilter(testdataset, 14, 'NUMERO_DEUDOR_PAIS_ID')
indexspaindata_original = getIndexFilter(testdataset_original, 'ES', 'NUMERO_DEUDOR_PAIS_ID')
print(indexspaindata)
print(indexspaindata_original)

In [123]:
display(testdataset.loc[indexspaindata[0]])
display(testdataset_original.loc[indexspaindata_original[0]])
display(testdataset_original[(testdataset_original['PRODUCTO_ID'] == 10189) & (testdataset_original['NUMERO_DEUDOR_PAIS_ID'] == 'ES') & (test)].index)
display(testdataset_original.loc[[644943]])
i=0
for v in indexspaindata_original:
    if v == 644943:
        break
    else:
        i = i+1
print(i)


In [101]:
explainer = lime.lime_tabular.LimeTabularExplainer(x_train.values, feature_names=x_train.columns, verbose=True, mode='regression')
exp = explainer.explain_instance(np.ravel(x_test.iloc[[indexspaindata[0]]]), nn_reg_pred, num_features=10)
exp.show_in_notebook(show_table=True)
exp.as_pyplot_figure()

In [102]:
exp.as_list()

In [68]:
display(testdataset.iloc[[100]])
display(y_pred[100])

### LSTM

In [8]:
torch.manual_seed(39931191)
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 5) #fully connected 1
        self.fc = nn.Linear(5, num_classes) #fully connected last layer
        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0.detach(), c_0.detach())) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Densetarget.squeeze(1)
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [9]:
num_epochs = 2000 #1000 epochs
learning_rate = 0.01 #0.001 lr

input_size = 25 #number of features
hidden_size = 20 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes

x_train_tensors = Variable(torch.Tensor(x_train.values))
x_test_tensors = Variable(torch.Tensor(x_test.values))
y_train_tensors = Variable(torch.Tensor(y_train.values))
y_test_tensors = Variable(torch.Tensor(y_test.values))

x_train_tensors_final = torch.reshape(x_train_tensors,   (x_train_tensors.shape[0], 1, x_train_tensors.shape[1]))
x_test_tensors_final = torch.reshape(x_test_tensors,  (x_test_tensors.shape[0], 1, x_test_tensors.shape[1]))

In [10]:
lstm_model = LSTM(num_classes, input_size, hidden_size, num_layers, x_train_tensors_final.shape[1]) #our lstm class

In [11]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=learning_rate)

In [12]:
for epoch in range(num_epochs):
  outputs = lstm_model.forward(x_train_tensors_final) #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors)
  loss.backward() #calculates the loss of the loss function
  optimizer.step() #improve from loss, i.e backprop
  if epoch % 100 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [13]:
train_predict = lstm1(x_test_tensors_final)#forward pass
data_predict = train_predict.data.numpy() #numpy conversion

In [ ]:
def predictplot(dso, dsp, dso_size, range_plot):
    plt.axvline(x=dso_size, c='r', linestyle='--') #size of the training set
    plt.scatter(range(0, range_plot),y_test[:range_plot], label='Actual Data') #actual plot
    plt.scatter(range(0, range_plot), data_predict[:range_plot], label='Predicted Data') #predicted plot
    plt.title('Time-Series Prediction')
    plt.legend()
    plt.savefig('../Output/timeseriespredictionLSTM.png')
    plt.show()

In [14]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, data_predict))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, data_predict))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, data_predict)))
print('R2:', metrics.r2_score(y_test, data_predict))

In [15]:
ig = IntegratedGradients(lstm1)
t = torch.Tensor(x_test.iloc[[20]].values)
x_test_tensors_final = torch.reshape(x_test_tensors,  (x_test_tensors.shape[0], 1, x_test_tensors.shape[1]))
x_test_tensors_final_ig = torch.reshape(x_test_tensors_final[0],  (1, 1, 25))
attribution = ig.attribute(x_test_tensors_final_ig)
x_test_tensors_final_ig
display(attribution.detach().numpy()[0][0])

In [16]:
def visualize_importance(title, features, importance, xtitle):
    plt.barh(range(0, features.size), importance, align = 'center')
    plt.yticks(range(0, features.size), features, wrap = True)
    plt.ylabel(xtitle)
    plt.title(title)
    plt.show()

In [18]:
visualize_importance('Variable Importance', x_test.columns, attribution.detach().numpy()[0][0], 'Feature')

In [51]:
lc = LayerConductance(lstm1.forward, layer, device_ids=None)

## Gaussian process regression

In [ ]:
#params = {'n_neighbors':[2,3,4,5,6,7,8,9]}

#knn = neighbors.KNeighborsRegressor()

#model = GridSearchCV(knn, params, cv=10)
#model.fit(x_train,y_train)
#model.best_params_

In [22]:
data_gaussian_filtered = getValuesFilter(data_gaussian, 14, data_gaussian.columns) # iso = * -> no filter by numero deudor pais id
data_gaussian_filtered = data_gaussian_filtered.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
traindataset, testdataset = train_test_split(data_gaussian, test_size=0.01, shuffle= False)
traindataset, testdataset = train_test_split(testdataset, test_size=0.01, shuffle= False)
x_train = traindataset.loc[:, traindataset.columns != 'IMP_VENTA_NETO_EUR']
y_train = traindataset.loc[:, traindataset.columns == 'IMP_VENTA_NETO_EUR']
x_train = x_train.drop(columns=['EDAD_RANGO_COMPRA'])
x_test = testdataset.loc[:, testdataset.columns != 'IMP_VENTA_NETO_EUR']
y_test = testdataset.loc[:, testdataset.columns == 'IMP_VENTA_NETO_EUR']
x_test = x_test.drop(columns = 'EDAD_RANGO_COMPRA')
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)
traindataset.shape

In [23]:
# massive dataset, try with something smaller
kernel = kernel = RBF(10, (1e-2, 1e2)) # Check which ones choose
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gpr.fit(x_train, y_train)
gpr.score(x_train, y_train)
y_pred, sigma = gpr.predict(x_test, return_std=True)
results['gaussianP'] = y_pred

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))